In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

%pwd

'/home/cseadmin/dz/TrafficFlowModel/data_process'

In [13]:
# 获取taxi中所有文件，之后读取使用
root_path = os.walk("../data/taxi")
all_files = []
for root,ds,fs in root_path:
    days = []
    for f in fs:
        full_path = os.path.join(root,f)
        days.append(full_path)
    if len(days)==0:
        continue
    all_files.append(days)
all_files

[['../data/taxi/202006/2020-06-04/2020-06-04_6',
  '../data/taxi/202006/2020-06-04/2020-06-04_0',
  '../data/taxi/202006/2020-06-04/2020-06-04_7',
  '../data/taxi/202006/2020-06-04/2020-06-04_2',
  '../data/taxi/202006/2020-06-04/2020-06-04_8',
  '../data/taxi/202006/2020-06-04/2020-06-04_5',
  '../data/taxi/202006/2020-06-04/2020-06-04_4',
  '../data/taxi/202006/2020-06-04/2020-06-04_9',
  '../data/taxi/202006/2020-06-04/2020-06-04_1',
  '../data/taxi/202006/2020-06-04/2020-06-04_3'],
 ['../data/taxi/202006/2020-06-06/2020-06-06_6',
  '../data/taxi/202006/2020-06-06/2020-06-06_8',
  '../data/taxi/202006/2020-06-06/2020-06-06_2',
  '../data/taxi/202006/2020-06-06/2020-06-06_7',
  '../data/taxi/202006/2020-06-06/2020-06-06_3',
  '../data/taxi/202006/2020-06-06/2020-06-06_9',
  '../data/taxi/202006/2020-06-06/2020-06-06_5',
  '../data/taxi/202006/2020-06-06/2020-06-06_0',
  '../data/taxi/202006/2020-06-06/2020-06-06_4',
  '../data/taxi/202006/2020-06-06/2020-06-06_1'],
 ['../data/taxi/20

In [5]:
df_temp=pd.read_csv("../data/taxi/202006/2020-06-01/2020-06-01_0", names=[
    "sys_time", "license_number", "lng", "lat", "gps_time", "EMPTY1", "speed",
    "direction", "car_status", "alarm_status", "EMPTY2", "EMPTY3",
    "license_color", "recorder_speed", "mileage", "height", "EMPTY4"
], nrows=100)

df_temp

,sys_time,license_number,lng,lat,gps_time,EMPTY1,speed,direction,car_status,alarm_status,EMPTY2,EMPTY3,license_color,recorder_speed,mileage,height,EMPTY4
0,2020-06-01 00:00:01,粤BD99912,113.986810,22.529696,2020-05-31 23:59:48,NaN,0,40,0,0,NaN,NaN,蓝色,0,2081590,0,0
1,2020-06-01 00:00:01,粤BDN1735,113.962010,22.536120,2020-05-31 23:59:49,NaN,0,0,0,0,NaN,NaN,蓝色,0,686220,0,0
2,2020-06-01 00:00:01,粤BDK8146,114.042880,22.598593,2020-05-31 22:22:57,NaN,0,173,0,0,NaN,NaN,蓝色,0,1894000,0,0
3,2020-06-01 00:00:01,粤BD84546,0.000000,0.000000,2020-05-31 23:59:49,NaN,0,0,0,32,NaN,NaN,蓝色,0,2484210,0,0
4,2020-06-01 00:00:01,粤BW7D07,0.000000,0.000000,2000-01-01 00:00:00,NaN,0,0,0,0,NaN,NaN,蓝色,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2020-06-01 00:00:01,粤BW5D72,0.000000,0.000000,2000-01-01 00:00:00,NaN,0,0,0,0,NaN,NaN,蓝色,0,0,0,0
96,2020-06-01 00:00:01,粤BDA8416,113.806870,22.666400,2020-05-31 23:59:49,NaN,28,0,512,0,NaN,NaN,蓝色,28,2751730,0,0
97,2020-06-01 00:00:01,粤BDK5138,114.119220,22.554642,2020-05-31 23:59:12,NaN,38,104,512,0,NaN,NaN,蓝色,38,2276880,0,0
98,2020-06-01 00:00:01,粤BDD0687,114.117744,22.540020,2020-05-31 23:59:51,NaN,47,121,512,0,NaN,NaN,蓝色,47,2702050,0,0


In [14]:
def notify(msg):
    import datetime
    channel="J0budaR2THarZw0OqS5O"
    notify_url=f"https://notify.run/{channel}"
    massage=f"{msg} | {str(datetime.datetime.now())}"
    os.system(f'curl {notify_url} -d "{massage}"')

In [20]:
names = [
    "sys_time", "license_number", "lng", "lat", "gps_time", "EMPTY1", "speed",
    "direction", "car_status", "alarm_status", "EMPTY2", "EMPTY3",
    "license_color", "recorder_speed", "mileage", "height", "EMPTY4"
]

# 这两个日期是需要处理的文件日期开头以及结尾+1
open_day = pd.to_datetime("2020-06-01 00:00:00")
close_day = pd.to_datetime("2020-07-01 00:00:00")

# 将数据进行初步处理
# dest_path是中间结果，暂存到一个文件夹中，之后读取再进行处理
dest_path = "../data/taxi_after_proc/intermediate/"
if not os.path.exists(dest_path):
    os.mkdir(dest_path)

all_files.sort()
for each_day in tqdm(all_files):
    each_day.sort()

    for each_time in tqdm(each_day):
        records = pd.read_csv(each_time, names=names)
        group = records.groupby("license_number")
        for each_lincense, each_record in group:
            posi = each_record[["lat", "lng", "gps_time", "speed"]]

            # 按gps时间排序，并去重
            posi["gps_time"] = pd.to_datetime(posi["gps_time"])
            posi.sort_values(by="gps_time", inplace=True, ascending=True)
            posi = posi.drop_duplicates(subset="gps_time", keep="first")

            posi = posi.loc[
                # 去除 GPS 0 值
                (posi["lat"] > 0) & (posi["lng"] > 0) &
                # 去除错误日期的记录
                (posi["gps_time"] > open_day) & (posi["gps_time"] < close_day) &
                # 保留速度 0~150 的记录
                (posi["speed"] > 0) & (posi["speed"] < 150)]

            # 去除数量过少的记录
            if len(posi) < 30:
                continue

            # 将结果输出到 dest_path 中
            posi.to_pickle(dest_path + each_time[-7:] + each_lincense + ".pkl")
            
notify("Finish intermediate.")

100%|██████████| 2/2 [44:38<00:00, 1339.05s/it]


In [22]:
res_path="../data/taxi_after_proc/clean202006/"
if not os.path.exists(res_path):
    os.mkdir(res_path)

files = os.listdir(dest_path)
car_set = set()

for file in files:
    car = file[7:-4]
    car_set.add(car)

days = []
for i in range(1, 31):
    days.append(str(i).zfill(2))

for car in car_set:
    for day in days:
        df_all = pd.DataFrame()
        for i in range(11):
            try:
                df_all = df_all.append(
                    pd.read_pickle(
                        "../data/taxi_after_proc/intermediate/06-{}_{}{}.pkl".
                        format(day, i, car)))
                df_all = df_all.drop_duplicates(subset="gps_time",
                                                keep="first")
                df_all = df_all[(df_all["gps_time"] > open_day)
                                & (df_all["gps_time"] < close_day)]
                df_all = df_all[(df_all["speed"] > 0)
                                & (df_all["speed"] < 150)]
#               print(df_all)
            except FileNotFoundError:
                continue
        df_all.to_pickle(
            "../data/taxi_after_proc/clean202006/06-{}_{}.pkl".format(day, car))
        
notify("Finish 202006.")

In [35]:
df1=pd.read_pickle("../data/taxi_after_proc/intermediate/06-01_9粤BDK8462.pkl")

df1

,lat,lng,gps_time,speed,direction,car_status
1138,22.529840,113.988510,2020-06-01 22:25:48,27,151,512
361286,22.524740,113.987915,2020-06-01 22:29:57,1,131,512
158093,22.523907,113.985270,2020-06-01 22:30:28,36,127,512
246819,22.523754,113.984570,2020-06-01 22:30:38,4,128,512
346398,22.523602,113.983760,2020-06-01 22:33:18,2,130,512
...,...,...,...,...,...,...
5815391,0.000000,0.000000,2020-06-01 23:57:48,55,0,0
5844283,22.579906,114.058815,2020-06-01 23:58:18,68,74,0
5866657,22.574550,114.060900,2020-06-01 23:58:38,57,84,0
5876607,22.573364,114.061066,2020-06-01 23:58:48,38,85,0


In [37]:
df2=pd.read_pickle("../data/taxi_after_proc/clean202006/06-01_粤BDK8462.pkl")

df2

,lat,lng,gps_time,speed,direction,car_status
17289,22.537394,114.108000,2020-06-01 00:00:08,72,125,512
89015,22.538422,114.097110,2020-06-01 00:01:18,65,134,512
126828,22.537992,114.090340,2020-06-01 00:01:56,70,126,512
141181,22.537586,114.088234,2020-06-01 00:02:08,59,131,512
163426,22.537460,114.084680,2020-06-01 00:02:29,72,134,512
...,...,...,...,...,...,...
5815391,0.000000,0.000000,2020-06-01 23:57:48,55,0,0
5844283,22.579906,114.058815,2020-06-01 23:58:18,68,74,0
5866657,22.574550,114.060900,2020-06-01 23:58:38,57,84,0
5876607,22.573364,114.061066,2020-06-01 23:58:48,38,85,0


In [2]:
df2=pd.read_pickle("../data/taxi_after_proc/clean202006/06-01_粤BDK8462.pkl")

df2

,lat,lng,gps_time,speed
17289,22.537394,114.108000,2020-06-01 00:00:08,72
89015,22.538422,114.097110,2020-06-01 00:01:18,65
126828,22.537992,114.090340,2020-06-01 00:01:56,70
141181,22.537586,114.088234,2020-06-01 00:02:08,59
163426,22.537460,114.084680,2020-06-01 00:02:29,72
...,...,...,...,...
5690989,22.593925,114.044876,2020-06-01 23:55:48,57
5844283,22.579906,114.058815,2020-06-01 23:58:18,68
5866657,22.574550,114.060900,2020-06-01 23:58:38,57
5876607,22.573364,114.061066,2020-06-01 23:58:48,38
